# Machine Learning Data Processing using Apache Spark 

---


This notebook shows the implementation of [Spark](https://spark.apache.org/), a Big Data Framework, for large-scale data processing, through an example using MLlib.

## Why Spark over Hadoop?

- Hadoop stores intermediate results onto the disk while processing data.
  - This is slower than how Spark operates, as the data moves to and fro from the memory to the disk.

- Spark stores the intermediate results onto the memory(unless it is full) while processing data.
  - This is faster than Hadoop, as the data stays in the memory(whenever possible).

- Spark follows Lazy Execution scheme to return outputs.
  - It first creates a Directed Acyclic Graph(DAG) to find the optimal path to return the output.

## How Big-Data is Processed?

- Given some big data, we can divide it into small chunks and distribute the chunks amongst a cluster of distributed systems(known as nodes).
-  The nodes in the cluster process these chunks and the fnal results are collected.

## MapReduce Paradigm for Big-Data

- MapReduce is a programming technique for manipulating large data sets.
  - "Hadoop MapReduce" is a specific implementation of this programming technique.
- The technique works by first dividing up a large dataset and distributing the data across a cluster.
- Three main steps in MapReduce:

<img src="https://raw.githubusercontent.com/deveshSingh06/Repo_Resources/master/Big_Data_Technologies/Images/Map_Reduce/0.png" width=600>
<br>
1. Map Step: In the map step, each data is analyzed and converted into a (key, value) pair.

<img src="https://raw.githubusercontent.com/deveshSingh06/Repo_Resources/master/Big_Data_Technologies/Images/Map_Reduce/1.png" width=600>
<br>      
2. Shuffle Step: Then these key-value pairs are shuffled across the cluster so that all keys are on the same machine.

<img src="https://raw.githubusercontent.com/deveshSingh06/Repo_Resources/master/Big_Data_Technologies/Images/Map_Reduce/2.png" width=600>
<br>   
3. Reduce step: In the reduce step, the values with the same keys are combined together.

<img src="https://raw.githubusercontent.com/deveshSingh06/Repo_Resources/master/Big_Data_Technologies/Images/Map_Reduce/2.png" width=600>
   
Image Credits: [Udacity](https://www.udacity.com/)




In [24]:
# SparkContext connects the cluster with the application
# SparkSession is Spark SQL's equivalent, used to read dataframes

from pyspark.sql import SparkSession

spark= SparkSession.builder.appName('Customers').getOrCreate()

In [25]:
from pyspark.ml.regression import LinearRegression


---

The dataset consists of information about consumers of a an video streaming app. We are supposed to predict how much they spend yearly for their services.

In [26]:
dataset=spark.read.csv("Data.csv",inferSchema=True,header=True)

In [27]:
dataset

DataFrame[Email: string, Address: string, Avg Session Length: double, Time on App: double, Time on Website: double, Length of Membership: double, Yearly Amount Spent: double]

Since Spark follows Lazy Execution, it doesn't return output untill it is forced to do so using show() or collect().

In [28]:
dataset.show()

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.72128268|         3.120178783|         581.852344|
|mstephens@davidso...|14023 Rodriguez P...|       33.33067252|12.79518855|  

In [29]:
dataset.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [30]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [31]:
featureassembler=VectorAssembler(inputCols=["Avg Session Length","Time on App","Time on Website",
                                            "Length of Membership"],outputCol="Independent Features")

In [32]:
output=featureassembler.transform(dataset)

In [33]:
output.show()

+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+--------------------+
|               Email|             Address|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|Independent Features|
+--------------------+--------------------+------------------+-----------+---------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|       34.49726773|12.65565115|    39.57766802|         4.082620633|         587.951054|[34.49726773,12.6...|
|   hduke@hotmail.com|4547 Archer Commo...|       31.92627203|11.10946073|    37.26895887|         2.664034182|        392.2049334|[31.92627203,11.1...|
|    pallen@yahoo.com|24645 Valerie Uni...|       33.00091476|11.33027806|    37.11059744|         4.104543202|        487.5475049|[33.00091476,11.3...|
|riverarebecca@gma...|1414 David Throug...|       34.30555663|13.71751367|    36.7

In [34]:
output.select("Independent Features").show()

+--------------------+
|Independent Features|
+--------------------+
|[34.49726773,12.6...|
|[31.92627203,11.1...|
|[33.00091476,11.3...|
|[34.30555663,13.7...|
|[33.33067252,12.7...|
|[33.87103788,12.0...|
|[32.0215955,11.36...|
|[32.73914294,12.3...|
|[33.9877729,13.38...|
|[31.93654862,11.8...|
|[33.99257277,13.3...|
|[33.87936082,11.5...|
|[29.53242897,10.9...|
|[33.19033404,12.9...|
|[32.38797585,13.1...|
|[30.73772037,12.6...|
|[32.1253869,11.73...|
|[32.33889932,12.0...|
|[32.18781205,14.7...|
|[32.61785606,13.9...|
+--------------------+
only showing top 20 rows



In [35]:
output.columns

['Email',
 'Address',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent',
 'Independent Features']

In [36]:
finalized_data=output.select("Independent Features","Yearly Amount Spent")

In [37]:
finalized_data.show()

+--------------------+-------------------+
|Independent Features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.49726773,12.6...|         587.951054|
|[31.92627203,11.1...|        392.2049334|
|[33.00091476,11.3...|        487.5475049|
|[34.30555663,13.7...|         581.852344|
|[33.33067252,12.7...|         599.406092|
|[33.87103788,12.0...|        637.1024479|
|[32.0215955,11.36...|        521.5721748|
|[32.73914294,12.3...|        549.9041461|
|[33.9877729,13.38...|         570.200409|
|[31.93654862,11.8...|        427.1993849|
|[33.99257277,13.3...|        492.6060127|
|[33.87936082,11.5...|        522.3374046|
|[29.53242897,10.9...|        408.6403511|
|[33.19033404,12.9...|        573.4158673|
|[32.38797585,13.1...|        470.4527333|
|[30.73772037,12.6...|        461.7807422|
|[32.1253869,11.73...|        457.8476959|
|[32.33889932,12.0...|        407.7045475|
|[32.18781205,14.7...|        452.3156755|
|[32.61785606,13.9...|        605.0610388|
+----------

In [38]:
train_data,test_data=finalized_data.randomSplit([0.75,0.25])

In [39]:
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Yearly Amount Spent')
regressor=regressor.fit(train_data)

In [40]:
pred_results=regressor.evaluate(test_data)

In [42]:
pred_results.predictions.show()

+--------------------+-------------------+------------------+
|Independent Features|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[30.57436368,11.3...|        442.0644138| 441.4843490309056|
|[30.81620065,11.8...|        266.0863409| 283.0946099202408|
|[30.83643267,13.1...|        467.5019004|  471.483692084249|
|[31.04722214,11.1...|        392.4973992|  387.478600236735|
|[31.26064687,13.2...|        421.3266313| 421.9273081093545|
|[31.30919264,11.9...|        432.7207178|  429.699260150114|
|[31.3123496,11.68...|         463.591418| 444.1484760429262|
|[31.36621217,11.1...|        430.5888826| 426.6102834910139|
|[31.42522688,13.2...|        530.7667187|    534.2557431369|
|[31.44597248,12.8...|        484.8769649| 481.5534946735463|
|[31.44744649,10.1...|        418.6027421| 425.9279908377571|
|[31.51473786,12.5...|         489.812488| 494.4400911047319|
|[31.52575242,11.3...|        443.9656268| 449.2893759606534|
|[31.526